In [11]:

import pandas as pd
from numpy.random import random
from chatbot import QWENChatBot
from pandarallel import pandarallel
import time
import random
from concurrent.futures import ThreadPoolExecutor
from typing import List

In [22]:
data = pd.read_excel('raw_data/LV_20250101-0206.xlsx')
data.head()

/Users/jiayi.wu/lvmhxaliyun/.venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,id,数据类型,分析对象ID,分析对象,站点id,站点名称,标题,内容,视频封面内容识别,音频内容识别,...,发布帖子地址,视频时长,图片上的标签列表,笔记里的标签列表,笔记中艾特的用户列表,合作品牌,用户地址,认证简介,MCN机构名称,用户所获点赞和收藏数
0,678fb7a6000000002900fbe6,电商笔记,58153241,分析对象1,146510,小红书,LV村上隆👟,#LV #村上隆 #三彩#LV鞋,① 小红,NaN,...,NaN,66.0,NaN,LV|村上隆|三彩|LV鞋,NaN,NaN,浙江台州,NaN,NaN,4207.0
1,677aaace000000000900ffc0,电商笔记,58153241,分析对象1,146510,小红书,🇸🇬LV & Murakami 🛍️🧢,lv✖️村上隆快闪店终于来新加坡啦！#新加坡lv #lvsg #lv快闪新加坡 #lv村上隆,①小红书 LOUISYUITTON MURAKAMI|② IMAXAJUM|③ LO...,NaN,...,NaN,NaN,NaN,新加坡lv|lvsg|lv快闪新加坡|lv村上隆,NaN,NaN,NaN,NaN,NaN,500.0
2,6783c74d0000000018013eb0,电商笔记,58153241,分析对象1,146510,小红书,分享｜2025 LV 村上隆,上次聯名已經是20年前了 這是專櫃目前有的現貨 SA又傳來了🤣🤣 但這次我沒買大包 只買一些...,① |② 小红,NaN,...,NaN,NaN,NaN,LV|村上隆|lv2025新款|村上隆太阳花|村上隆LV|louisvuitton|lv新品,NaN,NaN,台灣TPE,NaN,NaN,200.0
3,679cdd69000000002902b993,电商笔记,58153241,分析对象1,146510,小红书,NaN,出丁程鑫小卡 标价不包，多收可刀#时代少年团丁程鑫 #丁程鑫#时代少年团#周边出物 #丁程鑫...,① 转转@yaya,NaN,...,NaN,NaN,NaN,时代少年团丁程鑫|丁程鑫|时代少年团|周边出物|丁程鑫lv村上隆,NaN,NaN,NaN,NaN,NaN,NaN
4,677bd897000000002001f679,电商笔记,58153241,分析对象1,146510,小红书,免费参观！新加坡LVx村上隆快闪店排队现状,Louis Vuitton x Murakami Pop-up Store 路易威登x村上隆...,NaN,NaN,...,NaN,28.0,NaN,新加坡|新加坡旅游|新加坡lv|lv村上隆|lv快闪|lv快闪新加坡|lv|新加坡lv快闪|...,NaN,NaN,新加坡,旅行博主,NaN,243388.0


In [23]:
#content null
data = data[data['内容'].notnull()]

In [24]:
system_message = """
你是一名数据标注专家，现在需要对用户发布的关于 LVMH 产品及品牌的小红书帖子进行分类。你的任务是根据以下规则，将帖子分类为四种类型之一：

规则：
1. 如果帖子是营销贴/广告贴，请标注为 **1**。
   - 特征：以推广产品或鼓励购买为目的，包含促销、价格、优惠信息、购买链接或显而易见的商业宣传语言。
   - 特征：使用品牌合作或官方话术，明确商业性质内容。
2. 如果帖子与明星相关性较高，请标注为 **2**。
   - 特征：明星代言分析、明星街拍讨论、明星推广行为等
   - 特征：明确提到某明星（如“张某某代言”、明星机场图）
3. 如果帖子是普通用户对 LVMH 相关产品的真实反馈，请标注为 **0**。
   - 特征：分享个人真实体验或意见，通常语气更加自然，没有强烈的推销目的。
   - 特征：通常描述个人对产品的使用感受、优缺点评价。
4. 如果无法分类，请标注为 **-1**。
要求：
输出结果必须是整数，范围严格为 -1、0、1 或 2，不允许出现范围以外的任何内容，不要添加解释。
"""
chatbot = QWENChatBot(system_message=system_message)

'''
# 定义一个函数用于将帖子分类
def classify_post(content: str) -> dict:
    # 调用 QWEN 模型，得到响应
    try:
        response = chatbot.ask(content)
    except Exception as e:
        response = f"Error: {str(e)}"
    
    return response
'''
def classify_post_thread_safe(content: str) -> int:
    try:
        return chatbot.ask(content)
    except Exception as e:
        return f"Error: {e}"
def classify_posts_concurrently(posts: List[str], max_workers: int = 5) -> List[int]:
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 将所有帖子分配给线程池并行处理
        futures = list(executor.map(classify_post_thread_safe, posts))
        results.extend(futures)
    return results

# 使用线程池处理 DataFrame 数据
data['classification'] = classify_posts_concurrently(data['内容'].tolist(), max_workers=16)


'''
# 批量分类：为每一行调用 classify 方法
data['classification'] = data['内容'].apply(classify_post)
'''
data

,id,数据类型,分析对象ID,分析对象,站点id,站点名称,标题,内容,视频封面内容识别,音频内容识别,...,视频时长,图片上的标签列表,笔记里的标签列表,笔记中艾特的用户列表,合作品牌,用户地址,认证简介,MCN机构名称,用户所获点赞和收藏数,classification
0,678fb7a6000000002900fbe6,电商笔记,58153241,分析对象1,146510,小红书,LV村上隆👟,#LV #村上隆 #三彩#LV鞋,① 小红,NaN,...,66.0,NaN,LV|村上隆|三彩|LV鞋,NaN,NaN,浙江台州,NaN,NaN,4207.0,-1
1,677aaace000000000900ffc0,电商笔记,58153241,分析对象1,146510,小红书,🇸🇬LV & Murakami 🛍️🧢,lv✖️村上隆快闪店终于来新加坡啦！#新加坡lv #lvsg #lv快闪新加坡 #lv村上隆,①小红书 LOUISYUITTON MURAKAMI|② IMAXAJUM|③ LO...,NaN,...,NaN,NaN,新加坡lv|lvsg|lv快闪新加坡|lv村上隆,NaN,NaN,NaN,NaN,NaN,500.0,1
2,6783c74d0000000018013eb0,电商笔记,58153241,分析对象1,146510,小红书,分享｜2025 LV 村上隆,上次聯名已經是20年前了 這是專櫃目前有的現貨 SA又傳來了🤣🤣 但這次我沒買大包 只買一些...,① |② 小红,NaN,...,NaN,NaN,LV|村上隆|lv2025新款|村上隆太阳花|村上隆LV|louisvuitton|lv新品,NaN,NaN,台灣TPE,NaN,NaN,200.0,1
3,679cdd69000000002902b993,电商笔记,58153241,分析对象1,146510,小红书,NaN,出丁程鑫小卡 标价不包，多收可刀#时代少年团丁程鑫 #丁程鑫#时代少年团#周边出物 #丁程鑫...,① 转转@yaya,NaN,...,NaN,NaN,时代少年团丁程鑫|丁程鑫|时代少年团|周边出物|丁程鑫lv村上隆,NaN,NaN,NaN,NaN,NaN,NaN,1
4,677bd897000000002001f679,电商笔记,58153241,分析对象1,146510,小红书,免费参观！新加坡LVx村上隆快闪店排队现状,Louis Vuitton x Murakami Pop-up Store 路易威登x村上隆...,NaN,NaN,...,28.0,NaN,新加坡|新加坡旅游|新加坡lv|lv村上隆|lv快闪|lv快闪新加坡|lv|新加坡lv快闪|...,NaN,NaN,新加坡,旅行博主,NaN,243388.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6542,67923229000000001800a731,电商笔记,58153241,分析对象1,146510,小红书,🍅事事如意柿子挂件🎁,各位小可爱们，看过来👀！ \t 今天给大家安利一款超级萌的水果毛绒球挂件——柿子🍅！它不仅外...,①好柿发生 小红书 柿子挂件，好运连连 送礼自用，满满心意|②好稀发生 小红书,NaN,...,NaN,NaN,朱砂车挂|lv联名村上隆|香蕉猫|1965年蛇本命年佩戴|小红书春节乐市集,NaN,NaN,NaN,NaN,NaN,NaN,1
6543,679219dd000000001800dfda,电商笔记,58153241,分析对象1,146510,小红书,speedy30黑三彩上身测评｜谁说女生背着大,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,①黑三彩speedy3o 不同尺寸上身测评 160cmvs165cmvs170cmvs175...,NaN,...,NaN,NaN,speedy|lv|lv包|黑三彩|lv黑三彩|speedy30|lvspeedy30|lv...,NaN,NaN,广东深圳,寄卖店,NaN,141943.0,0
6544,678208f3000000001703b1fa,电商笔记,58153241,分析对象1,146510,小红书,NaN,感谢@路易威登 邀请品牌大使@时代少年团-丁程鑫 拍摄LV史上最具影响力的艺术家合作、风靡时...,①感谢路易威登 邀请品牌大使丁程鑫 你猜为什么只拍他 你年爹可不是自叫的 钢丁程鑫 LV村上...,NaN,...,14.0,NaN,丁程鑫|路易威登,NaN,NaN,NaN,NaN,NaN,NaN,2
6545,677a06be000000001300a5f8,电商笔记,58153241,分析对象1,146510,小红书,🇸🇬#村上隆LV,place,① MANALAUN|② LOUISVUITTON MURAKAMI|③ 小红书|④ ...,NaN,...,NaN,NaN,NaN,NaN,NaN,地球的某一片红薯地,NaN,NaN,100.0,-1


In [25]:
data.classification.value_counts()

classification
1                                                                                                                                                                                                                                                                                                   2661
0                                                                                                                                                                                                                                                                                                   2627
-1                                                                                                                                                                                                                                                                                                   692
2                                                                                             

In [26]:
filtered_data = data[data.classification == '0']
filtered_data

,id,数据类型,分析对象ID,分析对象,站点id,站点名称,标题,内容,视频封面内容识别,音频内容识别,...,视频时长,图片上的标签列表,笔记里的标签列表,笔记中艾特的用户列表,合作品牌,用户地址,认证简介,MCN机构名称,用户所获点赞和收藏数,classification
14,679260ba000000001800b050,电商笔记,58153241,分析对象1,146510,小红书,🇬🇧Recently plog🍛🥞💆🏻‍,1️⃣📍Drury 188-189 在holborn附近的网红brunch，点的开心果pan...,②小红书 HEYTEA|③POPHUART 延迟过万圣 POPMART Halloeen...,NaN,...,NaN,NaN,留学英国|发点生活边角料|清清日常库存|留学plog|伦敦brunch|labubu,NaN,NaN,广东广州,NaN,NaN,21000.0,0
15,6791f0c50000000029029680,电商笔记,58153241,分析对象1,146510,小红书,阿米的新年六位数购物开箱来了🎁,拖延症患者终于来开箱了，攒了一个多月的购物&护肤品分享，还有收到的礼物开箱，不算很齐全塞不下...,①UITTON LcUSvUmo 小红书 UITTON |②DUISVUITTON LO...,NaN,...,NaN,NaN,购物开箱分享|LV|lv村上隆|miumiu|chanel|新年礼物|乘光lumihood|...,NaN,NaN,中国,NaN,NaN,58000.0,0
16,6779f53f000000000b017934,电商笔记,58153241,分析对象1,146510,小红书,lv 村上隆限时店～还有2天！,一共三个店，黑彩，白彩，咖啡店～ 说实话三个店都非常可爱！但人真的好多！！（凑热闹本人+1）...,① LOUISVUITTON MURAKAMI|② ...,NaN,...,NaN,NaN,时尚潮人打卡地|随处走走就是潮流|潮流新地标,NaN,NaN,海外美国,NaN,NaN,71045.0,0
19,6784e9b10000000019034092,电商笔记,58153241,分析对象1,146510,小红书,终于到手了❗黑三彩carryall黑武士不要太好,开箱めLv村上隆包包■黑三彩Carryall 联名款黑武士羊皮carryall开箱分享 真的...,① 小红书 LOUISVUITON PARS,NaN,...,54.0,NaN,lv村上隆|LV黑三彩|lvcarryall|carryal新款|carryall|lv新款...,NaN,NaN,NaN,NaN,NaN,NaN,0
20,677953cd0000000013018d00,电商笔记,58153241,分析对象1,146510,小红书,Lv村上隆限定✖️白三彩卡包🆚黑三彩钱包,好可爱呀！兼并了美貌和实用#卡包推荐[话题]##钱包推荐[话题]# #lv[话题]# #l...,NaN,NaN,...,16.0,NaN,卡包推荐|钱包推荐|lv|lv村上隆联名|村上隆|lv白三彩|lv黑三彩,NaN,NaN,日本 东京,NaN,NaN,600.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6531,67867210000000001a013776,电商笔记,58153241,分析对象1,146510,小红书,这双鞋太好看啦！！,800年没买trainer的我陪母上大人逛街顺利蹭到一双[派对R][吐舌头H][耶R] #鞋...,NaN,NaN,...,NaN,NaN,鞋子分享|是心动的鞋款|lv村上隆|trainer,NaN,NaN,地球的某一片红薯地,NaN,NaN,0.0,0
6536,67778fed000000000b016bd1,电商笔记,58153241,分析对象1,146510,小红书,花花的包,村上隆太阳花熊猫。还有比这个组合更合适的包给我们花花吗[派对R][派对R][派对R]#LV ...,③小红书,NaN,...,NaN,NaN,LV|村上隆|lv村上隆手袋|大熊猫花花|lv新款|lv包,NaN,NaN,地球的某一片红薯地,NaN,NaN,200.0,0
6539,677aaefb0000000009014c0f,电商笔记,58153241,分析对象1,146510,小红书,NaN,#lv村上隆快闪店 太出片啦！！！今天下午去的，除了咖啡店拍的时间久点外，其他都很快，地方也...,①小红书 2003.01.09|②小红书 2003.01.09|④,NaN,...,NaN,NaN,lv村上隆快闪店|lv村上隆2025|跨年|ccd拍照,NaN,NaN,地球的某一片红薯地,NaN,NaN,20.0,0
6543,679219dd000000001800dfda,电商笔记,58153241,分析对象1,146510,小红书,speedy30黑三彩上身测评｜谁说女生背着大,lv村上隆联名这次出的speedy系列，比speedy30小的只有speedynano，有看...,①黑三彩speedy3o 不同尺寸上身测评 160cmvs165cmvs170cmvs175...,NaN,...,NaN,NaN,speedy|lv|lv包|黑三彩|lv黑三彩|speedy30|lvspeedy30|lv...,NaN,NaN,广东深圳,寄卖店,NaN,141943.0,0


In [27]:
filtered_data.to_excel('processed_data/raw_data_filtered.xlsx',index = False)